In [2]:
import os
import xml.dom.minidom as dom
import numpy as np
import pandas as pd

In [3]:
with open('../00_Download_and_Preprocessing/bam_files.txt', 'r') as f:
    bam_files = [os.path.basename(fn) for fn in f.read().strip().split('\n')]
htz1_rep1_rep2_bam_dict = {
    'SWR1 (WT)': [bam_files[0], bam_files[3]],
    'swc3 (LH∆)': [bam_files[1], bam_files[4]],
    'swc3∆': [bam_files[2], bam_files[5]]
}
swr1_bam_dict = {
    'SWR1 (WT)': [bam_files[6]],
    'swc3 (LH∆)': [bam_files[7]],
    'swc3∆': [bam_files[8]]
}
strain_color_dict = {
    'SWR1 (WT)': '#BFBFBF',
    'swc3 (LH∆)': '#D357FE',
    'swc3∆': '#4152F6'
}

In [5]:
def sliding_window(arr, window):
    width = np.shape(arr)[1]
    val = np.sum(arr[:, :window], axis=1) / window
    smoothed_arr = np.zeros((2, width - window + 1))
    smoothed_arr[:, 0] = val
    for i in range(1, width - window + 1):
        val += (arr[:, window + i - 1] - arr[:, i - 1]) / window
        smoothed_arr[:, i] = val
    return smoothed_arr

In [8]:
width = 1100
smoothing = 31
svgw = 300
svgh = 300
svgx = (40, 280)
svgy = (30, 265)
zero_idx = (width - smoothing + 1) // 2

def xscale(x):
    return (x + 500) / 1000 * (svgx[1] - svgx[0]) + svgx[0]

xvals = xscale(np.arange(-500, 500))

In [9]:
def create_axes(document, svg):
    xlabel = svg.appendChild(document.createElement('text'))
    xlabel.setAttribute('font-size', '12')
    xlabel.setAttribute('x', str((svgx[0] + svgx[1]) // 2))
    xlabel.setAttribute('y', str(svgh - 5))
    xlabel.setAttribute('text-anchor', 'middle')
    xlabel.appendChild(document.createTextNode('Distance from +1 Nucleosome'))
    xminlabel = svg.appendChild(document.createElement('text'))
    xminlabel.setAttribute('font-size', '12')
    xminlabel.setAttribute('x', str(svgx[0]))
    xminlabel.setAttribute('y', str(svgh - svgy[0] + 15))
    xminlabel.setAttribute('text-anchor', 'middle')
    xminlabel.appendChild(document.createTextNode('-500'))
    xmaxlabel = svg.appendChild(document.createElement('text'))
    xmaxlabel.setAttribute('font-size', '12')
    xmaxlabel.setAttribute('x', str(svgx[1]))
    xmaxlabel.setAttribute('y', str(svgh - svgy[0] + 15))
    xmaxlabel.setAttribute('text-anchor', 'middle')
    xmaxlabel.appendChild(document.createTextNode('500'))
    x0label = svg.appendChild(document.createElement('text'))
    x0label.setAttribute('font-size', '12')
    x0label.setAttribute('x', str(xscale(0)))
    x0label.setAttribute('y', str(svgh - svgy[0] + 15))
    x0label.setAttribute('text-anchor', 'middle')
    x0label.appendChild(document.createTextNode('0'))
    ylabel = svg.appendChild(document.createElement('text'))
    ylabel.setAttribute('font-size', '12')
    ylabel.setAttribute('x', str(svgx[0][0] - 12))
    ylabel.setAttribute('y', str((svgy[0] + svgy[1]) / 2))
    ylabel.setAttribute('text-anchor', 'middle')
    ylabel.setAttribute('transform', 'rotate(-90 {} {})'.format(svgx[0][0] - 12, (svgy[0] + svgy[1]) / 2))
    ylabel.appendChild(document.createTextNode('Occupancy (AU)'))
    yminlabel = svg.appendChild(document.createElement('text'))
    yminlabel.setAttribute('font-size', '12')
    yminlabel.setAttribute('x', str(svgx[0][0] - 10))
    yminlabel.setAttribute('y', str(svgy[1]))
    yminlabel.setAttribute('text-anchor', 'middle')
    yminlabel.appendChild(document.createTextNode('-1'))
    ymaxlabel = svg.appendChild(document.createElement('text'))
    ymaxlabel.setAttribute('font-size', '12')
    ymaxlabel.setAttribute('x', str(svgx[0][0] - 10))
    ymaxlabel.setAttribute('y', str(svgy[0]))
    ymaxlabel.setAttribute('text-anchor', 'middle')
    ymaxlabel.appendChild(document.createTextNode('1'))

    bottom_axis_group = svg.appendChild(document.createElement('g'))
    xaxis = bottom_axis_group.appendChild(document.createElement('line'))
    xaxis.setAttribute('x1', str(svgx[0]))
    xaxis.setAttribute('y1', str(svgy[1]))
    xaxis.setAttribute('x2', str(svgx[1]))
    xaxis.setAttribute('y2', str(svgy[1]))
    xaxis.setAttribute('stroke', '#000000')
    xaxis.setAttribute('stroke-width', '1')
    for x in [-400, -300, -200, -100, 0, 100, 200, 300, 400]:
        tick = bottom_axis_group.appendChild(document.createElement('line'))
        tick.setAttribute('x1', str(xscale(x)))
        tick.setAttribute('y1', str(svgy[1]))
        tick.setAttribute('x2', str(xscale(x)))
        tick.setAttribute('y2', str(svgy[1] - 6))
        tick.setAttribute('stroke', '#000000')
        tick.setAttribute('stroke-width', '1')
    top_axis_group = svg.appendChild(bottom_axis_group.cloneNode(True))
    top_axis_group.setAttribute('transform', 'rotate(180 {} {})'.format((svgx[1] + svgx[1]) // 2, (svgy[0] + svgy[1]) / 2))
    middle_axis_group = svg.appendChild(bottom_axis_group.cloneNode(True))
    middle_axis_group.setAttribute('transform', 'translate(0 {})'.format((svgy[0] - svgy[1]) / 2))
    for tick in middle_axis_group.childNodes[1:]:
        tick.setAttribute('y1', str(svgy[1] + 6))

    left_axis_group = svg.appendChild(document.createElement('g'))
    yaxis = left_axis_group.appendChild(document.createElement('line'))
    yaxis.setAttribute('x1', str(svgx[0][0]))
    yaxis.setAttribute('y1', str(svgy[0]))
    yaxis.setAttribute('x2', str(svgx[0][0]))
    yaxis.setAttribute('y2', str(svgy[1]))
    yaxis.setAttribute('stroke', '#000000')
    yaxis.setAttribute('stroke-width', '1')
    right_axis_group = svg.appendChild(left_axis_group.cloneNode(True))
    right_axis_group.setAttribute('transform', 'rotate(180 {} {})'.format((svgx[0][1] + svgx[1][0]) // 2, (svgy[0] + svgy[1]) // 2))
    return document

In [ ]:
def create_nucleosomes(document, svg):
    nuc_group = svg.appendChild(document.createElement('g'))
    dh = nuc_group.appendChild(document.createElement('line'))
    dh.setAttribute('x1', str(svgx[0][0]))
    dh.setAttribute('y1', str(svgy[1] - 1.05 * (svgy[1] - svgy[0])))
    dh.setAttribute('x2', str(svgx[1][1]))
    dh.setAttribute('y2', str(svgy[1] - 1.05 * (svgy[1] - svgy[0])))
    dh.setAttribute('stroke', '#000000')
    dh.setAttribute('stroke-width', '3')
    dh.setAttribute('stroke-linecap', 'round')
    nuc = nuc_group.appendChild(document.createElement('g'))
    nuc_body = nuc.appendChild(document.createElement('circle'))
    nuc_body.setAttribute('cx', '0')
    nuc_body.setAttribute('cy', '0')
    nuc_body.setAttribute('r', '10')
    nuc_body.setAttribute('stroke', '#000000')
    nuc_body.setAttribute('stroke-width', '1')
    nuc_body.setAttribute('fill', 'url(#nuc-grad)')
    nuc_s1 = nuc.appendChild(document.createElement('line'))
    nuc_s1.setAttribute('x1', '-9.39')
    nuc_s1.setAttribute('y1', '3.42')
    nuc_s1.setAttribute('x2', '5')
    nuc_s1.setAttribute('y2', '-8.66')
    nuc_s1.setAttribute('stroke', '#000000')
    nuc_s1.setAttribute('stroke-width', '3')
    nuc_s1.setAttribute('stroke-linecap', 'round')
    nuc_s2 = nuc.appendChild(document.createElement('line'))
    nuc_s2.setAttribute('x1', '-5')
    nuc_s2.setAttribute('y1', '8.66')
    nuc_s2.setAttribute('x2', '9.39')
    nuc_s2.setAttribute('y2', '-3.42')
    nuc_s2.setAttribute('stroke', '#000000')
    nuc_s2.setAttribute('stroke-width', '3')
    nuc_s2.setAttribute('stroke-linecap', 'round')
    nuc.setAttribute('transform', 'translate({} {})'.format(xscale_m1(0), svgy[1] - 1.05 * (svgy[1] - svgy[0])))
    for x in [-150, -300, -450]:
        nuc = nuc_group.appendChild(nuc.cloneNode(True))
        nuc.setAttribute('transform', 'translate({} {})'.format(xscale_m1(x), svgy[1] - 1.05 * (svgy[1] - svgy[0])))
    for x in [0, 150, 300, 450]:
        nuc = nuc_group.appendChild(nuc.cloneNode(True))
        nuc.setAttribute('transform', 'translate({} {})'.format(xscale_p1(x), svgy[1] - 1.05 * (svgy[1] - svgy[0])))

    label_m1 = nuc_group.appendChild(document.createElement('g'))
    rect_m1 = label_m1.appendChild(document.createElement('rect'))
    rect_m1.setAttribute('x', str(xscale_m1(0) - 8))
    rect_m1.setAttribute('y', str(svgy[1] - 1.05 * (svgy[1] - svgy[0]) - 5.5))
    rect_m1.setAttribute('width', '16')
    rect_m1.setAttribute('height', '11')
    rect_m1.setAttribute('rx', '5')
    rect_m1.setAttribute('ry', '5')
    rect_m1.setAttribute('stroke', '#000000')
    rect_m1.setAttribute('stroke-width', '0.5')
    rect_m1.setAttribute('fill', '#FFFFFF')
    rect_m1.setAttribute('opacity', '0.8')
    text_m1 = label_m1.appendChild(document.createElement('text'))
    text_m1.setAttribute('x', str(xscale_m1(0)))
    text_m1.setAttribute('y', str(svgy[1] - 1.05 * (svgy[1] - svgy[0]) + 3.3))
    text_m1.setAttribute('font-size', '10')
    text_m1.setAttribute('text-anchor', 'middle')
    text_m1.appendChild(document.createTextNode('-1'))

    label_p1 = nuc_group.appendChild(document.createElement('g'))
    rect_p1 = label_p1.appendChild(document.createElement('rect'))
    rect_p1.setAttribute('x', str(xscale_p1(0) - 8))
    rect_p1.setAttribute('y', str(svgy[1] - 1.05 * (svgy[1] - svgy[0]) - 5.5))
    rect_p1.setAttribute('width', '16')
    rect_p1.setAttribute('height', '11')
    rect_p1.setAttribute('rx', '5')
    rect_p1.setAttribute('ry', '5')
    rect_p1.setAttribute('stroke', '#000000')
    rect_p1.setAttribute('stroke-width', '0.5')
    rect_p1.setAttribute('fill', '#FFFFFF')
    rect_p1.setAttribute('opacity', '0.8')
    text_p1 = label_p1.appendChild(document.createElement('text'))
    text_p1.setAttribute('x', str(xscale_p1(0)))
    text_p1.setAttribute('y', str(svgy[1] - 1.05 * (svgy[1] - svgy[0]) + 3.3))
    text_p1.setAttribute('font-size', '10')
    text_p1.setAttribute('text-anchor', 'middle')
    text_p1.appendChild(document.createTextNode('+1'))

    ndr_label = nuc_group.appendChild(document.createElement('text'))
    ndr_label.setAttribute('x', str((svgx[0][0] + svgx[1][1]) // 2))
    ndr_label.setAttribute('y', str(svgy[1] - 1.064 * (svgy[1] - svgy[0])))
    ndr_label.setAttribute('font-size', '12')
    ndr_label.setAttribute('text-anchor', 'middle')
    ndr_label.appendChild(document.createTextNode('NDR'))
    return document

In [ ]:
def write_svg(strain_bam_dict, target, ymax):
    for bam in strain_bam_dict['SWR1 (WT)']:
        

In [10]:
def write_svgs(strain_bam_dict, target):
    for bed in os.listdir('../data/TagPileup'):
        comp_dir = '../data/TagPileup/{}/Composites'.format(bed)
        comp_plot_dir = '../03_Composite_Results/{}'.format(bed[:-6])
        if not os.path.exists(comp_plot_dir):
            os.mkdir(comp_plot_dir)
    
        y_max = 0
        wt_comps = {
            '+1': np.zeros((2, 651)),
            '-1': np.zeros((2, 651))
        }
        for bam in strain_bam_dict['WT']:
            comp_p1 = sliding_window(pd.read_csv('{}/{}_{}.out'.format(p1_dir, bam[:-4], bed), sep='\t', index_col=0).values, smoothing)
            wt_comps['+1'] += comp_p1[:, zero_idx - 150:zero_idx + 501]
    
            comp_m1 = sliding_window(pd.read_csv('{}/{}_{}.out'.format(m1_dir, bam[:-4], m1_bed), sep='\t', index_col=0).values, smoothing)
            wt_comps['-1'] += comp_m1[:, zero_idx - 500:zero_idx + 151]
        wt_y_max = np.stack([np.max(wt_comps['-1'], axis=1), np.max(wt_comps['+1'], axis=1)])
            
        y_max = max(y_max, np.max(wt_comps['+1']), np.max(wt_comps['-1']))
        strain_y_max = {}
        strain_comps = {strain: {'+1': np.zeros((2, 651)), '-1': np.zeros((2, 651))} for strain in strain_bam_dict if strain != 'WT'}
        for strain in strain_comps:
            for bam in strain_bam_dict[strain]:
                comp_p1 = sliding_window(pd.read_csv('{}/{}_{}.out'.format(p1_dir, bam[:-4], bed), sep='\t', index_col=0).values, smoothing)
                strain_comps[strain]['+1'] += comp_p1[:, zero_idx - 150:zero_idx + 501]
    
                comp_m1 = sliding_window(pd.read_csv('{}/{}_{}.out'.format(m1_dir, bam[:-4], m1_bed), sep='\t', index_col=0).values, smoothing)
                strain_comps[strain]['-1'] += comp_m1[:, zero_idx - 500:zero_idx + 151]
    
            y_max = max(y_max, np.max(strain_comps[strain]['-1']), np.max(strain_comps[strain]['+1']))
            strain_y_max[strain] = np.stack([np.max(strain_comps[strain]['-1'], axis=1), np.max(strain_comps[strain]['+1'], axis=1)])
    
        y_max *= 1.1
        def yscale(y):
            return (-y + y_max) * (svgy[1] - svgy[0]) / (2 * y_max) + svgy[0]
    
        for strain in strain_comps:
            document = dom.Document()
            svg = document.appendChild(document.createElement('svg'))
            svg.setAttribute('xmlns', 'http://www.w3.org/2000/svg')
            svg.setAttribute('font-family', 'Helvetica')
            svg.setAttribute('viewBox', '0 0 {} {}'.format(svgw, svgh))
            svg.setAttribute('baseProfile', 'full')
    
            defs = svg.appendChild(document.createElement('defs'))
            nuc_grad = defs.appendChild(document.createElement('radialGradient'))
            nuc_grad.setAttribute('id', 'nuc-grad')
            stop1 = nuc_grad.appendChild(document.createElement('stop'))
            stop1.setAttribute('offset', '0%')
            stop1.setAttribute('stop-color', '#FFFFFF')
            stop2 = nuc_grad.appendChild(document.createElement('stop'))
            stop2.setAttribute('offset', '100%')
            stop2.setAttribute('stop-color', '#C7BBB0')
            wt_m1_grad = defs.appendChild(document.createElement('linearGradient'))
            wt_m1_grad.setAttribute('id', 'wt-m1-grad')
            wt_m1_grad.setAttribute('x1', '0%')
            wt_m1_grad.setAttribute('y1', '0%')
            wt_m1_grad.setAttribute('x2', '0%')
            wt_m1_grad.setAttribute('y2', '100%')
            stop1 = wt_m1_grad.appendChild(document.createElement('stop'))
            stop1.setAttribute('offset', '0%')
            stop1.setAttribute('stop-color', strain_color_dict['WT'])
            stop1.setAttribute('stop-opacity', '1')
            stop2 = wt_m1_grad.appendChild(document.createElement('stop'))
            stop2.setAttribute('offset', '{}%'.format(wt_y_max[0, 0] * 100 / (wt_y_max[0, 0] + wt_y_max[0, 1])))
            stop2.setAttribute('stop-color', strain_color_dict['WT'])
            stop2.setAttribute('stop-opacity', '0')
            stop3 = wt_m1_grad.appendChild(document.createElement('stop'))
            stop3.setAttribute('offset', '100%')
            stop3.setAttribute('stop-color', strain_color_dict['WT'])
            stop3.setAttribute('stop-opacity', '1')
            wt_p1_grad = defs.appendChild(document.createElement('linearGradient'))
            wt_p1_grad.setAttribute('id', 'wt-p1-grad')
            wt_p1_grad.setAttribute('x1', '0%')
            wt_p1_grad.setAttribute('y1', '0%')
            wt_p1_grad.setAttribute('x2', '0%')
            wt_p1_grad.setAttribute('y2', '100%')
            stop1 = wt_p1_grad.appendChild(document.createElement('stop'))
            stop1.setAttribute('offset', '0%')
            stop1.setAttribute('stop-color', strain_color_dict['WT'])
            stop1.setAttribute('stop-opacity', '1')
            stop2 = wt_p1_grad.appendChild(document.createElement('stop'))
            stop2.setAttribute('offset', '{}%'.format(wt_y_max[1, 0] * 100 / (wt_y_max[1, 0] + wt_y_max[1, 1])))
            stop2.setAttribute('stop-color', strain_color_dict['WT'])
            stop2.setAttribute('stop-opacity', '0')
            stop3 = wt_p1_grad.appendChild(document.createElement('stop'))
            stop3.setAttribute('offset', '100%')
            stop3.setAttribute('stop-color', strain_color_dict['WT'])
            stop3.setAttribute('stop-opacity', '1')
            strain_m1_grad = defs.appendChild(document.createElement('linearGradient'))
            strain_m1_grad.setAttribute('id', 'strain-m1-grad')
            strain_m1_grad.setAttribute('x1', '0%')
            strain_m1_grad.setAttribute('y1', '0%')
            strain_m1_grad.setAttribute('x2', '0%')
            strain_m1_grad.setAttribute('y2', '100%')
            stop1 = strain_m1_grad.appendChild(document.createElement('stop'))
            stop1.setAttribute('offset', '0%')
            stop1.setAttribute('stop-color', strain_color_dict[strain])
            stop1.setAttribute('stop-opacity', '1')
            stop2 = strain_m1_grad.appendChild(document.createElement('stop'))
            stop2.setAttribute('offset', '{}%'.format(strain_y_max[strain][0, 0] * 100 / (strain_y_max[strain][0, 0] + strain_y_max[strain][0, 1])))
            stop2.setAttribute('stop-color', strain_color_dict[strain])
            stop2.setAttribute('stop-opacity', '0')
            stop3 = strain_m1_grad.appendChild(document.createElement('stop'))
            stop3.setAttribute('offset', '100%')
            stop3.setAttribute('stop-color', strain_color_dict[strain])
            stop3.setAttribute('stop-opacity', '1')
            strain_p1_grad = defs.appendChild(document.createElement('linearGradient'))
            strain_p1_grad.setAttribute('id', 'strain-p1-grad')
            strain_p1_grad.setAttribute('x1', '0%')
            strain_p1_grad.setAttribute('y1', '0%')
            strain_p1_grad.setAttribute('x2', '0%')
            strain_p1_grad.setAttribute('y2', '100%')
            stop1 = strain_p1_grad.appendChild(document.createElement('stop'))
            stop1.setAttribute('offset', '0%')
            stop1.setAttribute('stop-color', strain_color_dict[strain])
            stop1.setAttribute('stop-opacity', '1')
            stop2 = strain_p1_grad.appendChild(document.createElement('stop'))
            stop2.setAttribute('offset', '{}%'.format(strain_y_max[strain][1, 0] * 100 / (strain_y_max[strain][1, 0] + strain_y_max[strain][1, 1])))
            stop2.setAttribute('stop-color', strain_color_dict[strain])
            stop2.setAttribute('stop-opacity', '0')
            stop3 = strain_p1_grad.appendChild(document.createElement('stop'))
            stop3.setAttribute('offset', '100%')
            stop3.setAttribute('stop-color', strain_color_dict[strain])
            stop3.setAttribute('stop-opacity', '1')
    
            wt_group = svg.appendChild(document.createElement('g'))
            wt_comp_m1 = wt_group.appendChild(document.createElement('polygon'))
            wt_points_m1_sense = [(x, yscale(y)) for x, y in zip(x_m1, wt_comps['-1'][0, :])]
            wt_points_m1_anti = [(x, yscale(-y)) for x, y in zip(np.flip(x_m1), np.flip(wt_comps['-1'][1, :]))]
            wt_comp_m1.setAttribute('points', ' '.join('{},{}'.format(*p) for p in wt_points_m1_sense + wt_points_m1_anti))
            wt_comp_m1.setAttribute('fill', 'url(#wt-m1-grad)')
            wt_comp_m1.setAttribute('stroke', '#FFFFFF')
            wt_comp_m1.setAttribute('stroke-width', '1')
            wt_outline_m1_sense = wt_group.appendChild(document.createElement('path'))
            wt_outline_m1_sense.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in wt_points_m1_sense))
            wt_outline_m1_sense.setAttribute('fill', 'none')
            wt_outline_m1_sense.setAttribute('stroke', '#000000')
            wt_outline_m1_sense.setAttribute('stroke-width', '0.5')
            wt_outline_m1_anti = wt_group.appendChild(document.createElement('path'))
            wt_outline_m1_anti.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in wt_points_m1_anti))
            wt_outline_m1_anti.setAttribute('fill', 'none')
            wt_outline_m1_anti.setAttribute('stroke', '#000000')
            wt_outline_m1_anti.setAttribute('stroke-width', '0.5')
            wt_comp_p1 = wt_group.appendChild(document.createElement('polygon'))
            wt_points_p1_sense = [(x, yscale(y)) for x, y in zip(x_p1, wt_comps['+1'][0, :])]
            wt_points_p1_anti = [(x, yscale(-y)) for x, y in zip(np.flip(x_p1), np.flip(wt_comps['+1'][1, :]))]
            wt_comp_p1.setAttribute('points', ' '.join('{},{}'.format(*p) for p in wt_points_p1_sense + wt_points_p1_anti))
            wt_comp_p1.setAttribute('fill', 'url(#wt-p1-grad)')
            wt_comp_p1.setAttribute('stroke', '#FFFFFF')
            wt_comp_p1.setAttribute('stroke-width', '1')
            wt_outline_p1 = wt_group.appendChild(document.createElement('path'))
            wt_outline_p1.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in wt_points_p1_sense))
            wt_outline_p1.setAttribute('fill', 'none')
            wt_outline_p1.setAttribute('stroke', '#000000')
            wt_outline_p1.setAttribute('stroke-width', '0.5')
            wt_outline_p1_anti = wt_group.appendChild(document.createElement('path'))
            wt_outline_p1_anti.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in wt_points_p1_anti))
            wt_outline_p1_anti.setAttribute('fill', 'none')
            wt_outline_p1_anti.setAttribute('stroke', '#000000')
            wt_outline_p1_anti.setAttribute('stroke-width', '0.5')
    
            strain_group = svg.appendChild(document.createElement('g'))
            strain_comp_m1 = strain_group.appendChild(document.createElement('polygon'))
            strain_points_m1_sense = [(x, yscale(y)) for x, y in zip(x_m1, strain_comps[strain]['-1'][0, :])]
            strain_points_m1_anti = [(x, yscale(-y)) for x, y in zip(np.flip(x_m1), np.flip(strain_comps[strain]['-1'][1, :]))]
            strain_comp_m1.setAttribute('points', ' '.join('{},{}'.format(*p) for p in strain_points_m1_sense + strain_points_m1_anti))
            strain_comp_m1.setAttribute('fill', 'url(#strain-m1-grad)')
            strain_comp_m1.setAttribute('stroke', '#FFFFFF')
            strain_comp_m1.setAttribute('stroke-width', '1')
            strain_outline_m1_sense = strain_group.appendChild(document.createElement('path'))
            strain_outline_m1_sense.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in strain_points_m1_sense))
            strain_outline_m1_sense.setAttribute('fill', 'none')
            strain_outline_m1_sense.setAttribute('stroke', '#000000')
            strain_outline_m1_sense.setAttribute('stroke-width', '0.5')
            strain_outline_m1_anti = strain_group.appendChild(document.createElement('path'))
            strain_outline_m1_anti.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in strain_points_m1_anti))
            strain_outline_m1_anti.setAttribute('fill', 'none')
            strain_outline_m1_anti.setAttribute('stroke', '#000000')
            strain_outline_m1_anti.setAttribute('stroke-width', '0.5')
            strain_comp_p1 = strain_group.appendChild(document.createElement('polygon'))
            strain_points_p1_sense = [(x, yscale(y)) for x, y in zip(x_p1, strain_comps[strain]['+1'][0, :])]
            strain_points_p1_anti = [(x, yscale(-y)) for x, y in zip(np.flip(x_p1), np.flip(strain_comps[strain]['+1'][1, :]))]
            strain_comp_p1.setAttribute('points', ' '.join('{},{}'.format(*p) for p in strain_points_p1_sense + strain_points_p1_anti))
            strain_comp_p1.setAttribute('fill', 'url(#strain-p1-grad)')
            strain_comp_p1.setAttribute('stroke', '#FFFFFF')
            strain_comp_p1.setAttribute('stroke-width', '1')
            strain_outline_p1 = strain_group.appendChild(document.createElement('path'))
            strain_outline_p1.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in strain_points_p1_sense))
            strain_outline_p1.setAttribute('fill', 'none')
            strain_outline_p1.setAttribute('stroke', '#000000')
            strain_outline_p1.setAttribute('stroke-width', '0.5')
            strain_outline_p1_anti = strain_group.appendChild(document.createElement('path'))
            strain_outline_p1_anti.setAttribute('d', 'M' + 'L'.join('{} {}'.format(*p) for p in strain_points_p1_anti))
            strain_outline_p1_anti.setAttribute('fill', 'none')
            strain_outline_p1_anti.setAttribute('stroke', '#000000')
            strain_outline_p1_anti.setAttribute('stroke-width', '0.5')
    
            ref_label = svg.appendChild(document.createElement('text'))
            ref_label.setAttribute('font-size', '12')
            ref_label.setAttribute('text-anchor', 'left')
            ref_label.setAttribute('transform', 'translate({} {})'.format(svgx[0][0] + 5, svgy[0] + 18))
            class_row = ref_label.appendChild(document.createElement('tspan'))
            class_row.setAttribute('x', '0')
            class_row.setAttribute('y', '0')
            class_row.appendChild(document.createTextNode(class_dict[bed[:-6]][0]))
            n_row = ref_label.appendChild(document.createElement('tspan'))
            n_row.setAttribute('x', '0')
            n_row.setAttribute('y', '12')
            n_row.appendChild(document.createTextNode('N = {}'.format(class_dict[bed[:-6]][1])))
    
            wt_label = svg.appendChild(document.createElement('text'))
            wt_label.setAttribute('font-size', '12')
            wt_label.setAttribute('text-anchor', 'left')
            wt_label.setAttribute('x', str(svgx[0][0] + 5))
            wt_label.setAttribute('y', str(svgy[0] + 42))
            wt_label.setAttribute('fill', '#999999')
            wt_label.appendChild(document.createTextNode('WT'))
    
            strain_label = svg.appendChild(document.createElement('text'))
            strain_label.setAttribute('font-size', '12')
            strain_label.setAttribute('text-anchor', 'left')
            strain_label.setAttribute('x', str(svgx[0][0] + 5))
            strain_label.setAttribute('y', str(svgy[0] + 54))
            strain_label.setAttribute('fill', strain_color_dict[strain])
            strain_label.appendChild(document.createTextNode(strain))
    
            create_axes(document, svg)
            
            with open('{}/{}_{}_{}.svg'.format(comp_plot_dir, target, bed[:-6], strain.replace('∆', 'd')), 'w') as f:
                document.writexml(f, addindent='    ', newl='\n')

In [11]:
write_svgs(htz1_rep1_rep2_bam_dict, 'Htz1')